# Default notebook

This default notebook is executed using Databricks Workflows as defined in resources/datakickstart_dabs_job.yml.

In [ ]:
from datakickstart_dabs import main

main.get_taxis().show(10)

In [ ]:
my_df = spark.createDataFrame([['test1', 1], ['test2', 2]], ['name', 'num'])
my_df.show(10)

In [ ]:
import pyspark.sql.functions as fn

def save_summary(df):
  df2 = df.select(
        df.tpep_pickup_datetime.cast("date").alias("pickup_date"), 
        df.pickup_zip, 
        df.trip_distance,
        df.fare_amount
        )
  df2.createOrReplaceTempView("trip_tmp")

  df_agg = spark.sql("""
          SELECT 
            pickup_date, 
            pickup_zip, 
            SUM(trip_distance) as trip_distance, 
            SUM(fare_amount) as fare_amount
          FROM trip_tmp
          GROUP BY pickup_date, pickup_zip
        """)
        
  df_agg.write.mode("overwrite").saveAsTable("main.datakickstart_dev.trip_summary")

df = main.get_taxis()
save_summary(df)